# **Chapter 46: Project 3 - Productivity/Task Management App**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Architect an **offline-first** application where the local database is the source of truth
- Implement bidirectional synchronization between local storage and cloud backend with conflict resolution
- Schedule and manage background tasks using `workmanager` for periodic sync and reminders
- Integrate with device calendars to sync task deadlines
- Build robust data export (JSON/CSV) and import features for user data portability
- Handle complex notification logic with action buttons (Mark as Done, Snooze)
- Manage background execution limitations on iOS and Android

---

## **Prerequisites**

- Completed Chapter 45: Project 2 - Social Media Feed (mastery of BLoC and complex state)
- Completed Chapter 22: Local Data Persistence (Drift/SQLite or Hive)
- Understanding of background execution constraints on mobile operating systems
- Familiarity with JSON serialization and file system operations (`path_provider`)
- Basic knowledge of calendar APIs on Android and iOS

---

## **46.1 Architecture Overview: Offline-First**

In an offline-first architecture, the application functions fully without an internet connection. The UI reads from and writes to a local database. A background synchronization service attempts to push local changes to the server and pull remote updates when connectivity is restored.

### **Data Flow Strategy**

1. **User Action**: User creates a task. UI optimistically updates the local database.
2. **Local Persistence**: Task is saved to local SQLite/Hive immediately.
3. **Sync Queue**: A "sync request" is added to a queue table.
4. **Background Sync**: A service detects network availability, processes the queue, and attempts to push changes to the server.
5. **Conflict Resolution**: If the server has a newer version of the data, a merge strategy (e.g., "Server Wins" or "Last Write Wins") is applied.
6. **UI Update**: The UI observes the local database stream and updates automatically (reactive programming).

### **Project Structure**

```
lib/
├── features/
│   ├── tasks/
│   │   ├── data/
│   │   │   ├── datasources/
│   │   │   │   ├── task_local_datasource.dart  # Drift/Hive
│   │   │   │   └── task_remote_datasource.dart # API
│   │   │   ├── repositories/
│   │   │   │   └── task_repository_impl.dart   # Coordinates local/remote
│   │   │   └── models/
│   │   ├── domain/
│   │   │   ├── entities/
│   │   │   │   └── task.dart
│   │   │   └── repositories/
│   │   │       └── task_repository.dart
│   │   └── presentation/
│   ├── sync/
│   │   ├── data/
│   │   │   ├── sync_queue.dart
│   │   │   └── sync_service.dart
│   │   └── logic/
```

---

## **46.2 Local Database Implementation (Drift)**

We use **Drift** (formerly Moor) for this project because it provides type-safe SQL, reactive queries (Streams), and migration support, which are essential for offline-first apps.

### **Database Definition**

```dart
// lib/features/tasks/data/datasources/task_local_datasource.dart
import 'package:drift/drift.dart';

part 'task_local_datasource.g.dart';

// Table definition
class Tasks extends Table {
  TextColumn get id => text()();
  TextColumn get title => text().withLength(min: 1, max: 200)();
  TextColumn get description => text().nullable()();
  DateTimeColumn get dueDate => dateTime().nullable()();
  BoolColumn get isCompleted => boolean().withDefault(const Constant(false))();
  IntColumn get priority => integer().withDefault(const Constant(0))(); // 0=Low, 1=Med, 2=High
  
  // Sync status
  DateTimeColumn get updatedAt => dateTime()();
  BoolColumn get isSynced => boolean().withDefault(const Constant(false))();
  BoolColumn get isDeleted => boolean().withDefault(const Constant(false))(); // Soft delete
  
  @override
  Set<Column> get primaryKey => {id};
}

// Database class
@DriftDatabase(tables: [Tasks])
class AppDatabase extends _$AppDatabase {
  AppDatabase(QueryExecutor e) : super(e);

  @override
  int get schemaVersion => 1;

  @override
  MigrationStrategy get migration {
    return MigrationStrategy(
      onCreate: (Migrator m) async {
        await m.createAll();
      },
      onUpgrade: (Migrator m, int from, int to) async {
        // Handle migrations here
      },
    );
  }
  
  // Reactive query: Get all active tasks
  Future<List<Task>> getAllTasks() => select(tasks).get();
  
  // Stream for UI updates
  Stream<List<Task>> watchAllTasks() {
    return (select(tasks)
      ..where((t) => t.isDeleted.equals(false))
      ..orderBy([(t) => OrderingTerm(expression: t.dueDate, mode: OrderingMode.asc)]))
      .watch();
  }
  
  // Insert or Update
  Future<void> upsertTask(Task task) => into(tasks).insertOnConflictUpdate(task);
  
  // Soft delete
  Future<void> softDeleteTask(String id) {
    return (update(tasks)..where((t) => t.id.equals(id))).write(
      TasksCompanion(
        isDeleted: Value(true),
        updatedAt: Value(DateTime.now()),
        isSynced: Value(false),
      ),
    );
  }
  
  // Get unsynced items
  Future<List<Task>> getUnsyncedTasks() {
    return (select(tasks)..where((t) => t.isSynced.equals(false))).get();
  }
  
  // Mark as synced
  Future<void> markAsSynced(String id, DateTime serverUpdatedAt) {
    return (update(tasks)..where((t) => t.id.equals(id))).write(
      TasksCompanion(
        isSynced: Value(true),
        updatedAt: Value(serverUpdatedAt),
      ),
    );
  }
}
```

**Explanation:**

- **Drift Tables**: Defined as Dart classes extending `Table`. Drift generates the boilerplate.
- **Sync Columns**: `isSynced` tracks if the local change has reached the server. `isDeleted` enables soft deletion (hiding from UI but keeping for sync).
- **Streams**: `watchAllTasks()` returns a `Stream`. The UI subscribes to this; whenever the database changes, the UI updates automatically without manual state management calls.
- **Upsert**: `insertOnConflictUpdate` handles both insert (new) and update (existing) logic seamlessly.

---

## **46.3 Synchronization Service**

The repository coordinates between the local database and remote API, handling offline logic.

### **Repository Implementation**

```dart
// lib/features/tasks/data/repositories/task_repository_impl.dart
import 'package:connectivity_plus/connectivity_plus.dart';

class TaskRepositoryImpl implements TaskRepository {
  final AppDatabase localDataSource;
  final TaskRemoteDataSource remoteDataSource;
  final Connectivity connectivity;

  TaskRepositoryImpl({
    required this.localDataSource,
    required this.remoteDataSource,
    required this.connectivity,
  });

  @override
  Stream<List<Task>> watchTasks() {
    // UI always watches the local database
    return localDataSource.watchAllTasks();
  }

  @override
  Future<void> createTask(Task task) async {
    // 1. Save locally immediately
    final localTask = task.copyWith(
      updatedAt: DateTime.now(),
      isSynced: false,
    );
    
    await localDataSource.upsertTask(localTask);
    
    // 2. Try to sync if online
    await _attemptSync(localTask);
  }

  Future<void> _attemptSync(Task task) async {
    final connectivityResult = await connectivity.checkConnectivity();
    
    if (connectivityResult != ConnectivityResult.none) {
      try {
        final serverTask = await remoteDataSource.createTask(task);
        
        // Update local record with server ID/timestamp and mark synced
        await localDataSource.markAsSynced(task.id, serverTask.updatedAt);
      } catch (e) {
        // If sync fails, keep the local record (it's already saved)
        // A background worker will retry later
        print('Sync failed: $e');
      }
    }
  }
  
  @override
  Future<void> syncPendingChanges() async {
    final unsynced = await localDataSource.getUnsyncedTasks();
    
    for (var task in unsynced) {
      try {
        if (task.isDeleted) {
          await remoteDataSource.deleteTask(task.id);
        } else {
          await remoteDataSource.createTask(task);
        }
        await localDataSource.markAsSynced(task.id, task.updatedAt);
      } catch (e) {
        // Stop sync on failure to prevent data loss
        rethrow;
      }
    }
  }
}
```

**Explanation:**

- **Reactive Stream**: `watchTasks()` provides the UI with data. The repository doesn't return `Future<List<Task>>` because the data might change due to background sync.
- **Optimistic Persistence**: `createTask` writes to the database *before* attempting network sync. This ensures the user sees their task immediately, even in airplane mode.
- **Sync Flag**: `isSynced: false` marks the record as "dirty". The background worker scans for these records later.
- **Connectivity Check**: `connectivity_plus` checks if the network is available before attempting the expensive HTTP call.

### **Background Synchronization with Workmanager**

We use `workmanager` to schedule periodic sync tasks (e.g., every 15 minutes) to reconcile local and server data.

```dart
// lib/services/background_sync.dart
import 'package:workmanager/workmanager.dart';

void callbackDispatcher() {
  Workmanager().executeTask((task, inputData) async {
    // Initialize dependency injection for background isolate
    configureDependencies();
    
    try {
      final repository = getIt<TaskRepository>();
      await repository.syncPendingChanges();
      
      // Fetch updates from server
      final serverTasks = await getIt<TaskRemoteDataSource>().fetchTasks();
      for (var serverTask in serverTasks) {
        // Compare updatedAt timestamps and merge
        // (Implementation omitted for brevity: logic checks if server is newer)
        await getIt<AppDatabase>().upsertTask(serverTask);
      }
      
      return Future.value(true);
    } catch (e) {
      return Future.value(false);
    }
  });
}

// Initialization in main.dart
void main() {
  Workmanager().initialize(
    callbackDispatcher,
    isInDebugMode: false,
  );
  
  // Register periodic task
  Workmanager().registerPeriodicTask(
    "sync-tasks",
    "sync-task-worker",
    frequency: const Duration(minutes: 15), // Minimum allowed by Android
    constraints: Constraints(
      networkType: NetworkType.connected,
    ),
  );
  
  runApp(MyApp());
}
```

**Explanation:**

- **callbackDispatcher**: The entry point for the background task. Runs in a separate isolate (no UI context).
- **registerPeriodicTask**: Schedules the task to run roughly every 15 minutes.
- **Constraints**: `NetworkType.connected` ensures the task only runs when the device has internet, saving battery.
- **Frequency Limitations**: Android imposes a minimum 15-minute interval for periodic work. iOS background fetch is less predictable and controlled by the OS.

---

## **46.4 Background Tasks and Notifications**

Scheduling exact-time notifications for reminders (e.g., "Meeting at 5 PM").

### **Notification Service**

```dart
// lib/services/notification_service.dart
import 'package:flutter_local_notifications/flutter_local_notifications.dart';

@singleton
class NotificationService {
  final FlutterLocalNotificationsPlugin _notifications = FlutterLocalNotificationsPlugin();

  Future<void> initialize() async {
    const androidSettings = AndroidInitializationSettings('@mipmap/ic_launcher');
    const iosSettings = DarwinInitializationSettings();
    
    const initSettings = InitializationSettings(
      android: androidSettings,
      iOS: iosSettings,
    );
    
    await _notifications.initialize(
      initSettings,
      onDidReceiveNotificationResponse: _onNotificationTapped,
    );
  }

  void _onNotificationTapped(NotificationResponse response) {
    final payload = response.payload;
    if (payload != null) {
      // Handle navigation based on payload
      // e.g., Navigate to TaskDetailScreen(taskId: payload)
    }
  }

  Future<void> scheduleTaskReminder(Task task) async {
    if (task.dueDate == null || task.dueDate!.isBefore(DateTime.now())) return;

    await _notifications.zonedSchedule(
      task.id.hashCode, // Unique int ID
      'Task Reminder',
      task.title,
      tz.TZDateTime.from(task.dueDate!, tz.local),
      const NotificationDetails(
        android: AndroidNotificationDetails(
          'task_reminders',
          'Task Reminders',
          channelDescription: 'Notifications for task due dates',
          importance: Importance.high,
          priority: Priority.high,
          actions: <AndroidNotificationAction>[
            AndroidNotificationAction(
              'mark_done',
              'Mark as Done',
              showsUserInterface: true,
            ),
          ],
        ),
        iOS: DarwinNotificationDetails(),
      ),
      androidScheduleMode: AndroidScheduleMode.exactAllowWhileIdle,
      uiLocalNotificationDateInterpretation: UILocalNotificationDateInterpretation.absoluteTime,
      payload: task.id,
    );
  }
  
  Future<void> cancelReminder(String taskId) async {
    await _notifications.cancel(taskId.hashCode);
  }
}
```

**Explanation:**

- **zonedSchedule**: Schedules a notification at a specific time (`TZDateTime`). Requires `android.permission.SCHEDULE_EXACT_ALARM` for exact timing on Android 12+.
- **Notification Actions**: `AndroidNotificationAction` adds buttons like "Mark as Done" directly in the notification shade. Tapping them triggers `onDidReceiveNotificationResponse`.
- **Timezone**: Requires `flutter_timezone` package to convert `DateTime` to the device's local timezone (`tz.local`).

---

## **46.5 Calendar Integration**

Allowing users to add tasks to their device calendar.

```dart
// lib/services/calendar_service.dart
import 'package:add_2_calendar/add_2_calendar.dart';

class CalendarService {
  Future<void> addTaskToCalendar(Task task) async {
    if (task.dueDate == null) return;

    final event = Event(
      title: task.title,
      description: task.description,
      location: null,
      startDate: task.dueDate!,
      endDate: task.dueDate!.add(const Duration(hours: 1)),
      allDay: false,
      iosParams: const IOSParams(
        reminder: Duration(minutes: 30),
      ),
      androidParams: const AndroidParams(
        emailInvites: [],
      ),
    );

    await Add2Calendar.addEvent2Cal(event);
  }
}

// Usage in UI
ElevatedButton.icon(
  icon: const Icon(Icons.calendar_today),
  label: const Text('Add to Calendar'),
  onPressed: () {
    getIt<CalendarService>().addTaskToCalendar(task);
  },
)
```

**Explanation:**

- **add_2_calendar**: A plugin that abstracts the Intent system on Android and EventKit UI on iOS. It opens the default calendar app with pre-filled data.
- **Permissions**: On Android, `ADD_EVENT` permissions are often not strictly required because the plugin launches the calendar app externally rather than writing directly. On iOS, no special entitlement is needed to *open* the calendar.

---

## **46.6 Data Export and Import**

Compliance with data privacy regulations often requires allowing users to export their data.

### **Export Logic**

```dart
// lib/features/settings/data/export_service.dart
import 'dart:io';
import 'package:path_provider/path_provider.dart';
import 'package:share_plus/share_plus.dart';

class ExportService {
  final AppDatabase database;

  ExportService(this.database);

  Future<File> exportTasksToJson() async {
    final tasks = await database.getAllTasks();
    
    // Convert to list of JSON maps
    final taskList = tasks.map((t) => {
      'id': t.id,
      'title': t.title,
      'description': t.description,
      'due_date': t.dueDate?.toIso8601String(),
      'is_completed': t.isCompleted,
      'priority': t.priority,
    }).toList();
    
    final json = jsonEncode({'tasks': taskList});
    
    // Write to file
    final directory = await getApplicationDocumentsDirectory();
    final file = File('${directory.path}/tasks_backup.json');
    await file.writeAsString(json);
    
    return file;
  }

  Future<void> shareExport() async {
    final file = await exportTasksToJson();
    
    await Share.shareXFiles(
      [XFile(file.path)],
      subject: 'Task Backup',
    );
  }
}
```

**Explanation:**

- **path_provider**: Finds a safe directory to write the file (`getApplicationDocumentsDirectory`).
- **share_plus**: Uses the native share sheet, allowing users to save the file to Drive, Email, or AirDrop. This avoids complex file permission handling for saving to "Downloads".
- **JSON Format**: A universal format that can be re-imported or used in other tools.

---

## **Chapter Summary**

In this chapter, we built a robust offline-first Task Management app:

### **Key Takeaways:**

1.  **Offline-First Architecture**: The UI subscribes to a local database stream (`watchTasks`). The network is a background detail for synchronization, not the primary data source.
2.  **Drift (Moor)**: A powerful SQL abstraction for Flutter that supports reactive queries, complex migrations, and type safety.
3.  **Sync Strategy**: Optimistic updates (save local immediately, sync later). Use `isSynced` flags to track "dirty" records.
4.  **Background Work**: `workmanager` handles periodic sync. Use constraints (`NetworkType.connected`) to save battery.
5.  **Exact Notifications**: `zonedSchedule` allows precise reminders. Handle notification actions (e.g., "Mark Done") via callbacks.
6.  **Calendar Integration**: Use intent-based plugins like `add_2_calendar` to avoid complex permission handling while providing seamless calendar functionality.

### **Offline-First Checklist:**
- ✅ App functions fully without internet
- ✅ Local database serves as source of truth
- ✅ Conflict resolution strategy defined (e.g., Server Wins)
- ✅ Soft deletion implemented (allows undo and proper sync deletion)
- ✅ Background sync runs only when network is available

---

## **Next Steps**

In the next chapter, **Chapter 47: Project 4 - Streaming/Media App**, we will explore the complexities of media playback. You'll learn how to integrate video players with DRM protection, manage background audio playback with lock screen controls, implement adaptive bitrate streaming (HLS/DASH), and build media casting support (Chromecast/AirPlay).

---

**End of Chapter 46**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='45. social_media_feed.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='47. streaming_media_app.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
